## Maximum Likelyhood for PGM

In [229]:
import tensorflow as tf
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Read in the graph

In [230]:
x = pd.read_csv('car.csv')


### Define the graph structure

In [231]:
# family = {'C':['A','B'], 'D':['C'],'F':['D','B','C']}
family = {'verdict':['buying','maint', 'doors', 'persons', 'lug_boot', 'safety']}

In [232]:
for child in family:
    print("child: ",child,"\nparents: ", family[child])

child:  verdict 
parents:  ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']


### Compute the maximum likelyhood counts

In [238]:
# dictionaries that will contain the data to calculate MLE for the specific graph structure   
childict = {}
childict_true = {}

for child in family:
    # list of all the parents for a specific child node
    parlist = [parent for parent in family[child]]
    # lists containing entries for all parent combinations as well as ones only for when the child is True
    partuplist = []
    partuplist_true = []
    # iterate through all entries and store the data in the lists
    checklist = []
    for item in x[child].values:
        checklist.append(item)
    for (check,item) in zip(checklist, x[[*parlist]].values):
        partuplist.append((tuple(parlist),tuple(item)))
        if check == 'vgood' or 'acc':
            partuplist_true.append((tuple(parlist),tuple(item)))
    
    # add the lists to the dictionaries
    childict[child] =partuplist
    childict_true[child] =partuplist_true
    
# get the set of all the parent combinations for each child (all other combinations will have probability 0 or Ebs)
childsetdict = {}
for child in childict:
    childsetdict[child] = set(childict[child])
# for child in childsetdict:
#     print("child: ",child,", Parents: ", childsetdict[child])
    
# dictionaries to hold the prelim counts
countdict = {}
countdict_true = {}
# initializing counts to 0
for child in childsetdict:
    for parents in childsetdict[child]:
        countdict[parents] = (0, child)
        countdict_true[parents] = (0, child)
# sum all occurances of each parent combination for each child   
for child in childict:
    for entry in childict[child]:
        countdict[entry] = (countdict[entry][0] + 1,child)
# sum all occurances of each parent combination for each child when the child is True
for child in childict_true:
    for entry in childict_true[child]:
        countdict_true[entry] = (countdict_true[entry][0] + 1,child)
# calculate MLE from the two counts above
mledict = {}       
for entry in countdict:
    mledict[entry] = (countdict_true[entry][0]/countdict[entry][0],countdict_true[entry][1])


### Print the MLE conditional probabilities for each child

In [240]:
# print the MLE conditional probabilities for each child
# table = np.empty([len(mledict)*2,len(x.columns)+1],dtype=str)
table = []

# i = 0
# for entry in mledict:
#     j = 0
#     for val in entry[1]:
#         table[i][j] = val 
#         print(table[i][j])
#         j = j + 1
#     table[i][j] =  1
#     table[i][1+j] =  mledict[entry][0]
#     i = i + 1
#     table[i][j] = 0
#     table[i][j+1] = 1 -  mledict[entry][0]
#     i = i + 1
    
for entry in mledict:
    row = []
    for val in entry[1]:
        row.append(val)
    row.append(val)
#     print(mledict[entry][0])
    row.append(mledict[entry][0])
    table.append(row)
    
# print(table[0:100,:])
#     print("P(",mledict[entry][1]," = 1 |",entry,") = ", mledict[entry][0])
#     print("P(",mledict[entry][1]," = 0 |",entry,") = ", 1-mledict[entry][0])
    

In [241]:
df = pd.DataFrame(data=table, columns=[c for c in x.columns] + ['condprob'])
df.to_csv('out.csv',sep=',')

In [211]:
# print(table[1000:1100,:])

## Calculate the prior probabilities

In [246]:
# find out how many categories each RV can take
rvcategorydict = {}
for RV in x.columns:
    rvcategorydict[RV] = tuple(set([item[0] for item in x[[RV]].values]))

### Print out the possible categories per RV

In [247]:
for RV in rvcategorydict:
    print(RV,rvcategorydict[RV])

verdict ('vgood', 'good', 'acc', 'unacc')
doors ('5more', '2', '3', '4')
safety ('med', 'low', 'high')
lug_boot ('med', 'small', 'big')
maint ('low', 'med', 'high', 'vhigh')
persons ('2', '4', 'more')
buying ('low', 'med', 'high', 'vhigh')


In [249]:
countsdict = {}

for RV in x.columns:
    countsdict[RV] = (rvcategorydict[RV],np.zeros(len(rvcategorydict[RV])))
#     for cat in rvcategorydict[RV]:
#         countsdict[rvcategorydict[RV]] = 0
        
for RV in x.columns:
    for item in x[[RV]].values:
        countsdict[RV] = countsdict[RV][1] + 1  
        
    